In [1]:
import numpy as np
import scipy.io as sio

import csv
import os
import json

from glob import glob
from collections import defaultdict

In [2]:
jsonFileName = './inputJson/test.json'

f = open(jsonFileName)
inputJson = json.load(f)
dataPathBase = inputJson['dataPathBase']
dataType = inputJson['dataType']
f.close()

print(dataType)
if dataType == 'survey':
    activities = ['bed', 'fall', 'pickup', 'run', 'sitdown', 'standup', 'walk']

survey


In [8]:
if dataType == "survey":
    dataPath = dataPathBase + "HAR_survey/"
    dataDict = {activity:[] for activity in activities}
    for activity in activities:
        dataDict[activity] = defaultdict(list)

elif dataType == "SHARP":
    dataPath = dataPathBase + "HAR_SHARP/"

for subFolder in os.walk(dataPath):
    if dataType == "survey":
        fileNameList = glob(subFolder[0] + "/input*.csv")
    print(len(fileNameList))
    
    for fileIndex, fileDirName in enumerate(fileNameList):
        fileName = fileDirName.split('/')[-1]
        fileType = fileName.split('_')[0]
        fileActv = fileName.split('_')[1]

        annotFileName = "annotation" + fileName[5:]
        annotFileDirName = subFolder[0] + annotFileName

        with open(fileDirName, mode='r') as file:
            csiData = list(csv.reader(file))
            csiDataNP = np.array(csiData)
            
        with open(annotFileDirName, mode='r') as annotFile:
            annotData = list(csv.reader(annotFile))
            annotDataNP = np.array(annotData)

        dataDict[fileActv]['CSI'].append(np.array([0]))
        dataDict[fileActv]['annot'].append(annotDataNP)

557


KeyError: 'input'